In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import math
import re
import csv
import json
import http.client
import requests
import shlex
import copy
import time
import datetime
import warnings
import sys
from datetime import datetime, timedelta, date
from io import StringIO
from itertools import product, chain
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'
warnings.filterwarnings('ignore')
# Note:
    # Download BSE, NSE, BFO AND NFO after 15:30:00
    # For CDS, run download after 17:00:00 only
    # For MCX, run MCX_Scheduler


In [ ]:

def updateTickersToExchangeFile():
    exchangeList = ['BFO', 'NFO','MCX', 'CDS'] 
    NFO_stockOptions = pd.read_csv('./extras/NFO_Sat.csv')['Symbol'].to_list()
    BSE = ['SENSEX', 'SNSX50', 'BANKEX']

    NSE = ['INDIA VIX','NIFTY 50','NIFTY BANK','NIFTY FIN SERVICE', 'NIFTY MID SELECT', 'NIFTY NEXT 50']
    NSE = [
            'NIFTY METAL', 'NIFTY IT', 'NIFTY ENERGY', 'NIFTY INFRA', 'NIFTY FMCG', 
            'NIFTY PHARMA', 'NIFTY AUTO', 'NIFTY MEDIA', 'NIFTY REALTY', 'NIFTY HEALTHCARE',
            'NIFTY PSE', 'NIFTY PSU BANK', 'NIFTY SERV SECTOR', 'NIFTY PVT BANK', 
            'NIFTY 200', 'NIFTY MNC', 'LTF', 'UNITDSPR', 'RPOWER', 'SCHNEIDER',
            'OLAELEC', 'NIFTY MIDCAP 100','NIFTY 100','NIFTY COMMODITIES',
            'NIFTY CONSUMPTION','NIFTY MIDCAP 50','NIFTY ALPHA 50','NIFTY MIDCAP 150',
            'NIFTY SMLCAP 50', 'NIFTY SMLCAP 100', 'NIFTY SMLCAP 250','NIFTY 500',
            'NIFTY CPSE', 'NIFTY NEXT 50','NIFTY MIDCAP SELECT (MIDCPNIFTY)',
            'NIFTY FINSRV25 50', 'NIFTY CONSR DURBL', 'NIFTY OIL AND GAS',
            'NIFTY500 MULTICAP', 'NIFTY LARGEMID250', 'NIFTY200MOMENTM30', 'NIFTY TOTAL MKT',
            'NIFTY MICROCAP250', 'NIFTY INDIA MFG', 'NIFTY IND DIGITAL', 'ARVIND', 'SANOFI',
            'IFBIND','IFCI', 'TATAINVEST', 'BECTORFOOD', 'IONEXCHANG', 'IPCALAB',
            'ITC', 'AWHCL', 'LIQUIDETF', 'KOPRAN', 'LICHSGFIN', 'IRFC', 'M&M',
            'MTNL', 'CONSUMBEES', 'CENTRUM', 'MIDHANI', 'ONGC', 'ORIENTHOT','ORIENTPPR',
            'PGHH', 'LEMONTREE', 'DIVOPPBEES', 'SAIL','ORIENTELEC', 'SIEMENS','BORORENEW',
            'HONDAPOWER', 'THOMASCOOK', 'PHARMABEES', 'AUTOBEES','SILVERBEES','MID150BEES',
            'NIFTYBEES', 'JUNIORBEES', 'LIQUIDBEES', 'BANKBEES', 'GOLDBEES', 'PSUBNKBEES',
            'SHARIABEES', 'LTGILTBEES', 'ITBEES', 'INFRABEES', 'NIF100BEES', 'GIFT NIFTY',
            'MAPMYINDIA', "TMCV", "ETERNAL", "GMRAIRPORT", "IDFCFIRSTB", "SAMMAANCAP", 
            "ARVIND", "AXISBANK", "PIRAMALFIN"
                
    ] + NFO_stockOptions  + NSE
    NSE = sorted(list(set(NSE)))
    BSE = sorted(list(set(BSE)))
    CDS = ['USDINR','DUSDINR','EURINR','EURUSD','JPYINR','GBPINR', 'ONMIBOR']
    BFO = ['SENSEX', 'BANKEX']
    NFO = ['BANKNIFTY', 'FINNIFTY', 'MIDCPNIFTY', 'NIFTY']
    MCX = ['CRUDEOIL', 'ALUMINIUM', 'COPPER', 'NICKEL', 'ZINC', 
           'LEAD','GOLD', 'SILVER', 'NATURALGAS', 
            'MCXGOLDEX', 'MCXMETLDEX', 'MCXCRUDEX', 'MCXCOPRDEX', 
           'MCXCOMPDEX','MCXBULLDEX', 'MCXENERGY', 'MCXAGRI', 'MCXMETAL', 'MCXSILVDEX']
    
    returnDict = dict()
    returnDict['BSE'] = BSE
    returnDict['NSE'] = NSE
    currentMonth ,currentYear, currentDay = datetime.now().month, datetime.now().year, datetime.now().day
     
    for exchange in exchangeList:
        zerodhaInstrumentsGetLink = 'https://api.kite.trade/instruments/' + exchange
        df = pd.read_csv(StringIO(requests.get(zerodhaInstrumentsGetLink).text))[[
            'instrument_token', 'tradingsymbol', 'name', 'expiry', 'segment', 'exchange'
            ]]
        df['name'].fillna(df['tradingsymbol'], inplace=True)
        df['expiry'] = pd.to_datetime(df['expiry'], format='%Y-%m-%d')
        df = df[
                ((df['expiry'].dt.month==currentMonth)
                & (df['expiry'].dt.year==currentYear)
                & (df['expiry'].dt.day== currentDay))
                | (df['expiry'].isna())
            ].reset_index()

        df = df[(df['name'].isin(MCX + CDS + BFO + NFO + NFO_stockOptions + NSE))]
        print(exchange, end= ' ')
        print(df['name'].unique(), end= ' ')
        
        
        if(df['expiry'].isna().all()):
            print("No expiry today")
            symbolList = [np.nan]
        else:
            try:
                print(f"Expiry : {df.reset_index(drop=True).loc[0]['expiry'].strftime('%d-%m-%Y')}")
            except:
                print(f"Expiry : {df[~(df['expiry'].isna())].reset_index(drop=True).loc[0]['expiry'].strftime('%d-%m-%Y')}")
            
            if(exchange=='CDS'):
                symbolList = CDS    
            else:
                symbolList = list(df['name'].unique())
                
        returnDict[exchange] = symbolList
        
    returnDf = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in returnDict.items()]))
    returnDf = returnDf[[ 'NSE', 'NFO','BSE', 'BFO', 'CDS', 'MCX']]
    
    print()
    print(returnDf)
    returnDf.to_csv('./tickers-to-download-all-exchanges.csv', index=False)

updateTickersToExchangeFile()



In [ ]:

# get kite connection constant

def getKiteConnectionConstants():
    # read curl request file and split all text with quotes
    curlRequest_text_file = open('./kite-curl-request.txt', 'r')
    curlCommandSplitList = shlex.split(curlRequest_text_file.read())
    curlRequest_text_file.close()
    requestUrl = ''
    headerParamsList = []
    
    # get request url and headers from file
    for i, requestParam in enumerate(curlCommandSplitList):
        if ( requestParam == 'curl' ):
            requestUrl = curlCommandSplitList[i+1]
        elif ( requestParam == '-H' ):
            keyValueTemp = curlCommandSplitList[i+1].split(': ')
            headerParamsList.append(keyValueTemp)
    headerParamsList = [item for sublist in headerParamsList for item in sublist]
    headerParamsDict = {headerParamsList[i]: headerParamsList[i + 1] for i in range(0, len(headerParamsList), 2)}
    
    # retrive domain, connection url
    kiteDomain, requestConnection = requestUrl.split('//')[1].split('/', 1)
    requestConnection = '/' +requestConnection
    requestConnectionList = requestConnection.split('&', 1)[0].split('/')
    
    return [kiteDomain, requestConnectionList, headerParamsDict]



In [ ]:
def fetchDataFromZerodha(row, kiteDomain, requestConnectionList, headerParamsDict):
    
    fromDate, toDate, expiryOfTicker, filePath = row['from_date'], row['to_date'], row['expiry'], row['file_path']
    ticker, tickerType, tickerExchange, instrumentToken, underlyingSymbol = row['tradingsymbol'], row['segment'], row['exchange'], str(row['instrument_token']), row['name']
    fileFullPath = filePath + ticker + '.csv'
        
    # read log file for selected exchange
    exchangeLogsFilePath = './logs/' + tickerExchange + '.csv'
    exchangeLogs = pd.read_csv(exchangeLogsFilePath)
    
    
    # create chunks of from_date and to_date
    fromToRange, chunkSize = [], 51
    startDate, stepDate = fromDate, fromDate+timedelta(days=chunkSize-1)
    allCandles = pd.DataFrame(columns = ['DateTime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest'])
    
    while( stepDate <= toDate ):
        fromToRange.append([startDate, stepDate])
        startDate, stepDate = stepDate+timedelta(days=1), stepDate+timedelta(days=chunkSize)
    if ( startDate <= toDate ):
        fromToRange.append([startDate, toDate])
   
    # set request connection prefix
    if ( requestConnectionList[4].isdigit() ):
        del requestConnectionList[4]
        requestConnectionList.insert(4, instrumentToken)
    
    requestConnectionPrefix = 'https://' + kiteDomain + ('/').join(requestConnectionList) + '&oi=1&from='
    
    print() 
    print(row.name+1, ticker, tickerType, instrumentToken)
    print('Expiry :: ', expiryOfTicker)
    
    for startDate, endDate in fromToRange:
        requestConnectionSufix = startDate.strftime('%Y-%m-%d') + '&to=' + endDate.strftime('%Y-%m-%d')
        requestConnection = requestConnectionPrefix + requestConnectionSufix
        kiteResponseData = requests.request('GET', requestConnection, headers=headerParamsDict, data={})
        maxRequestRetry, requestRetryCount = 5, 0
        
        # send get request to zerodha for maximum given times
        # If request fails, will not update Logs for the given Ticker
        while ( requestRetryCount < maxRequestRetry ):
            try:
                candles = pd.DataFrame(
                            json.loads(kiteResponseData.text)['data']['candles'], 
                            columns = ['DateTime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']
                          )
                print(requestConnectionSufix + ' : ' + str(len(candles)))
                allCandles = pd.concat([allCandles, candles])
                requestRetryCount = 5
                
            except:
                print(f'{requestConnectionSufix} : No Candles Found')
                requestRetryCount = requestRetryCount + 1
                kiteResponseData.close()
                time.sleep(1)
                kiteResponseData = requests.request('GET', requestConnection, headers=headerParamsDict, data={})
                if ( requestRetryCount == maxRequestRetry ):
                    print('Maximum Retries Limit Reached')
        
        kiteResponseData.close()
        time.sleep(1)
    
    
    # append data to existing file and update logs
    if ( len(allCandles) > 0 ):
        
        if not ( os.path.exists(filePath) ):
            os.makedirs(filePath)
        
        getDate = lambda dateAndTime: dateAndTime.split('T')[0]
        getTime = lambda dateAndTime: dateAndTime.split('T')[-1].split('+')[0]
        allCandles['Date'] = pd.to_datetime(allCandles['DateTime'].apply(lambda dateAndTime: getDate(dateAndTime)), format='%Y-%m-%d')
        allCandles['Time'] = pd.to_datetime(allCandles['DateTime'].apply(lambda dateAndTime: getTime(dateAndTime)), format='%H:%M:%S').dt.strftime('%H:%M:%S')
        allCandles['Ticker'] = ticker
        allCandles['Expiry Date'] = expiryOfTicker if len(tickerType.split('-'))==2 else np.nan
        allCandles = allCandles[['Ticker', 'Date', 'Time', 'Expiry Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']]
        
        if ( os.path.exists(fileFullPath) ):
            try:
                allCandles = pd.concat([
                    pd.read_csv(fileFullPath, parse_dates=['Date'], date_parser=(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))), 
                    allCandles
                ])

            except:
                try:
                    allCandles = pd.concat([
                        pd.read_csv(fileFullPath, parse_dates=['Date'], date_parser=(lambda x: pd.to_datetime(x, format='%d-%m-%Y'))), 
                        allCandles
                    ])
                except:
                    print("Mixed Values in Date Column of Original File")
                    old_df = pd.read_csv(fileFullPath)
                    old_df = old_df.reset_index(drop=True)
                    for p in range(0, len(old_df)):
                        try:
                            old_df.at[p, 'Date'] = pd.to_datetime(old_df.at[p, 'Date'], format='%Y-%m-%d')
                        except:
                            try:
                                old_df.at[p, 'Date'] = pd.to_datetime(old_df.at[p, 'Date'], format='%d-%m-%Y')
                            except:
                                pass
                    old_df = old_df[['Ticker', 'Date', 'Time', 'Expiry Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']]
                    allCandles = pd.concat([old_df,allCandles])

        
        # Drop Duplicates in case of exceptional cases
        # Save the file to its path
        allCandles = allCandles.drop_duplicates(
            subset=['Ticker', 'Date', 'Time'],
            keep='last'
        )
        allCandles = allCandles[['Ticker', 'Date', 'Time', 'Expiry Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']]
        allCandles.sort_values(['Date', 'Time'], inplace=True)
        allCandles.set_index('Ticker').to_csv(fileFullPath)
        
        
        # find oldest and newest date to update/add in exchange logs
        oldestDate, newestDate = allCandles['Date'].min(), allCandles['Date'].max()
        tickerCondition = (exchangeLogs['tradingsymbol'] == ticker) 
        
        # if we get old logs then update else append new logs for ticker
        if tickerCondition.any():
            if(len(exchangeLogs.loc[tickerCondition])>1):
                expiryCondition = (exchangeLogs.loc[tickerCondition]['expiry'].isin(allCandles['Expiry Date'].unique()))
                tickerCondition = (tickerCondition & expiryCondition)    
            exchangeLogs.loc[tickerCondition, 'to_date'] = newestDate.strftime('%Y-%m-%d')
            
        else:
            exchangeLogs = pd.concat([
                exchangeLogs, pd.DataFrame({
                    'tradingsymbol': [ticker],
                    'name': [underlyingSymbol],
                    'segment': [tickerType],
                    'expiry': [expiryOfTicker] if len(tickerType.split('-'))==2 else [np.nan],
                    'from_date': [oldestDate.strftime('%Y-%m-%d')],
                    'to_date': [newestDate.strftime('%Y-%m-%d')],
                    'prepared_from_date': [np.nan],
                    'prepared_to_date': [np.nan]
                })
            ], ignore_index=True)
        
        exchangeLogs.set_index('tradingsymbol').to_csv(exchangeLogsFilePath)
    
    # return str(instrumentToken)+'_Done'




In [ ]:
# Set exchange to download Tickers

In [ ]:
## # indices present in NFO zerodha
nfoIndicesName = ['NIFTY', 'BANKNIFTY', 'FINNIFTY', 'MIDCPNIFTY', 'NIFTYNXT50']

# location where we want to save all files
defaultSavePath = pd.read_csv('./location.csv')['Location'][0]

# choose which exchange's data we want to download
exchangeToDownload = ['NSE', 'NFO', 'BSE', 'BFO', 'CDS', 'MCX'][
    int(input('Choose Exchange to Download Data\n\t1. NSE\n\t2. NFO\n\t3. BSE\n\t4. BFO\n\t5. CDS\n\t6. MCX\nEnter Exchange Number :: ')) - 1
]

zerodhaInstrumentsGetLink = 'https://api.kite.trade/instruments/' + exchangeToDownload
zerodhaInstruments = pd.read_csv(StringIO(requests.get(zerodhaInstrumentsGetLink).text))[[
    'instrument_token', 'tradingsymbol', 'name', 'expiry', 'segment', 'exchange'
]]

zerodhaInstruments['name'].fillna(zerodhaInstruments['tradingsymbol'], inplace=True)

# read tickers we want to download under individual exchange
tickersToDownload = pd.read_csv('./tickers-to-download-all-exchanges.csv')[exchangeToDownload].dropna().to_list()

tickersUnderExchange = zerodhaInstruments[
    zerodhaInstruments['tradingsymbol' if (exchangeToDownload=='BSE' or exchangeToDownload=='NSE') else 'name'].isin(tickersToDownload)
].set_index('instrument_token')



# assign segment's name under exchange
if ( exchangeToDownload == 'BSE' or exchangeToDownload == 'NSE' ):
    tickersUnderExchange.loc[tickersUnderExchange['segment'] == exchangeToDownload, 'segment'] = 'STOCKS'
elif ( exchangeToDownload == 'NFO' ):
    tickersUnderExchange.loc[(tickersUnderExchange['segment'] == exchangeToDownload+'-FUT'), 'segment'] = 'STOCKS-FUTURE'
    tickersUnderExchange.loc[(tickersUnderExchange['segment'] == exchangeToDownload+'-OPT'), 'segment'] = 'STOCKS-OPTION'
    tickersUnderExchange.loc[
        (tickersUnderExchange['segment'] == 'STOCKS-FUTURE') & (tickersUnderExchange['name'].isin(nfoIndicesName)), 'segment'
    ] = 'INDICES-FUTURE'
    tickersUnderExchange.loc[
        (tickersUnderExchange['segment'] == 'STOCKS-OPTION') & (tickersUnderExchange['name'].isin(nfoIndicesName)), 'segment'
    ] = 'INDICES-OPTION'
else:
    tickersUnderExchange.loc[tickersUnderExchange['segment'] == exchangeToDownload+'-FUT', 'segment'] = 'INDICES-FUTURE'
    tickersUnderExchange.loc[tickersUnderExchange['segment'] == exchangeToDownload+'-OPT', 'segment'] = 'INDICES-OPTION'

# read logs file under exchange 
# and assign from_date and to_date as per available data
availableFiles = pd.read_csv('./logs/' + exchangeToDownload + '.csv')
availableFiles['to_date'] = pd.to_datetime(availableFiles['to_date'], format='%Y-%m-%d')+timedelta(days=1)
availableFiles = availableFiles.drop(columns=['from_date']).rename(columns={'to_date': 'from_date'})

tickersUnderExchange['from_date'] = pd.to_datetime(
    np.select([tickersUnderExchange['segment'].isin(['INDICES', 'STOCKS'])], [datetime(2015, 1, 1)], 
              default=date.today()-timedelta(days=100)), 
    format='%Y-%m-%d'
)

tickersUnderExchange.loc[tickersUnderExchange['segment'].isin(['INDICES', 'STOCKS']), 'name'] = tickersUnderExchange['tradingsymbol']

tickersUnderExchange = tickersUnderExchange.reset_index().merge(availableFiles, on=['tradingsymbol', 'name', 'expiry', 'segment'], how='left').set_index('instrument_token')

tickersUnderExchange['from_date_y'].fillna(tickersUnderExchange['from_date_x'], inplace=True)
tickersUnderExchange = tickersUnderExchange.drop(columns=['from_date_x']).rename(columns={'from_date_y': 'from_date'})
tickersUnderExchange['to_date'] = pd.to_datetime(date.today().strftime('%Y-%m-%d'), format='%Y-%m-%d')

# create dataframe for ticker not able to download under exchange
tickersDontUnderExchange = tickersUnderExchange[tickersUnderExchange['from_date'] > tickersUnderExchange['to_date']]
tickersDontUnderExchange.to_csv(f'./downloads/tickers-dont-download-under-exchange-{exchangeToDownload}.csv')

# Create Dataframe for ticker able to Download - sorted by Expiry to address prioritisation and avoidable data loss
tickersUnderExchange = tickersUnderExchange[tickersUnderExchange['from_date'] <= tickersUnderExchange['to_date']]
tickersUnderExchange['expiry'] = pd.to_datetime(tickersUnderExchange['expiry'], format='%Y-%m-%d', errors="coerce")
tickersUnderExchange = tickersUnderExchange.sort_values(
                            by=['expiry','segment','name', 'tradingsymbol'], 
                            na_position='first')

# Get current month expiry Data in Stock-Option on Expiry Day
# Relevant when downloading Stock Options Data along with others
tickersUnderExchange = tickersUnderExchange[    
                            (
                                (tickersUnderExchange['expiry'].dt.month == datetime.now().month) 
                                & (tickersUnderExchange['expiry'].dt.year == datetime.now().year)
                                & (tickersUnderExchange['expiry'].dt.day == datetime.now().day)
                            ) 
                            | (tickersUnderExchange['expiry'].isna()) 
                       ]

tickersUnderExchange['expiry'] = tickersUnderExchange['expiry'].dt.strftime('%Y-%m-%d')
tickersUnderExchange.to_csv(f'./downloads/tickers-to-download-under-exchange-{exchangeToDownload}.csv')

# create file path for individual tickers
filePathSufix = np.select(
    [tickersUnderExchange['segment'].isin(['INDICES-FUTURE', 'STOCKS-FUTURE']), tickersUnderExchange['segment'].isin(['INDICES-OPTION', 'STOCKS-OPTION'])], 
    [tickersUnderExchange['name'] + '/', tickersUnderExchange['name'] + '/' + tickersUnderExchange['expiry'] + '/'], 
    default=''
)
tickersUnderExchange['file_path'] = defaultSavePath + tickersUnderExchange['exchange'] + '/' + tickersUnderExchange['segment'] + '/' + filePathSufix

tickersUnderExchange
# print(tickersUnderExchange.shape)
# create zerodha kite connection
kiteDomain, requestConnectionList, headerParamsDict = getKiteConnectionConstants()



In [ ]:
# tickersUnderExchange = pd.read_csv("./downloads/tickers-to-download-under-exchange-NSE.csv")
# # print(tickersUnderExchange)
# filePathSufix = np.select(
#     [tickersUnderExchange['segment'].isin(['INDICES-FUTURE', 'STOCKS-FUTURE']), tickersUnderExchange['segment'].isin(['INDICES-OPTION', 'STOCKS-OPTION'])], 
#     [tickersUnderExchange['name'] + '/', tickersUnderExchange['name'] + '/' + tickersUnderExchange['expiry'] + '/'], 
#     default=''
# )
# tickersUnderExchange['file_path'] = defaultSavePath + tickersUnderExchange['exchange'] + '/' + tickersUnderExchange['segment'] + '/' + filePathSufix
# tickersUnderExchange['from_date'] = pd.to_datetime(tickersUnderExchange['from_date'], format='%Y-%m-%d')
# tickersUnderExchange['to_date'] = pd.to_datetime(tickersUnderExchange['to_date'], format='%Y-%m-%d')
# print(tickersUnderExchange)

MAX_RETRIES = 5
retries = 0
i = 0
df = tickersUnderExchange.reset_index().copy(deep=True)
while i < len(df):
    row = df.iloc[i]
    try:
        fetchDataFromZerodha(row, kiteDomain, requestConnectionList, headerParamsDict)
        i += 1
        retries = 0  # reset after success
    except Exception as e:
        retries += 1
        print(f"Error at index {i} (retry {retries}/{MAX_RETRIES}): {e}")

        if retries >= MAX_RETRIES:
            print(f"Skipping index {i} after max retries")
            i += 1
            retries = 0
print("All Done")
